In [9]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
from scipy import stats
from tensorflow.keras.callbacks import TensorBoard

In [2]:
actions = np.array(['Countrys_lockdown','Covid-19','Curfield','Fake_news','Favipiravir','Infected_patient','Work_from_home'])
no_sequences = 100
sequence_length = 50

In [3]:
label_map = {label:num for num, label in enumerate(actions)}
print(label_map)

{'Countrys_lockdown': 0, 'Covid-19': 1, 'Curfield': 2, 'Fake_news': 3, 'Favipiravir': 4, 'Infected_patient': 5, 'Work_from_home': 6}


In [4]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join('MP_Data', action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join('MP_Data', action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [5]:
print(np.array(sequences).shape)
print(np.array(labels).shape)

(700, 50, 1662)
(700,)


In [6]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.95)

In [10]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [8]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(256, return_sequences=True, activation='relu'))
model.add(LSTM(512, return_sequences=True, activation='relu'))
model.add(LSTM(512, return_sequences=False, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

#checkpoint = ModelCheckpoint(filepath='best_weights.h5', save_best_only=True, save_weights_only=True)
#lr_reduce = ReduceLROnPlateau(monitor='loss', factor=0.3, patience=2, verbose=2, mode='max')
#early_stop = EarlyStopping(monitor='loss', min_delta=0.1, patience=1, mode='min')

In [19]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_21 (LSTM)               (None, 50, 64)            442112    
_________________________________________________________________
batch_normalization (BatchNo (None, 50, 64)            256       
_________________________________________________________________
lstm_22 (LSTM)               (None, 50, 64)            33024     
_________________________________________________________________
lstm_23 (LSTM)               (None, 50, 128)           98816     
_________________________________________________________________
batch_normalization_1 (Batch (None, 50, 128)           512       
_________________________________________________________________
lstm_24 (LSTM)               (None, 50, 64)            49408     
_________________________________________________________________
lstm_25 (LSTM)               (None, 50, 128)          

In [18]:
from keras.utils.vis_utils import plot_model
plot_model(model)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [20]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback], steps_per_epoch=1, validation_steps=1)

Epoch 1/2000
1/1 [==============================] - 4s 4s/step - loss: 0.2210 - accuracy: 0.9849
Epoch 2/2000
1/1 [==============================] - 2s 2s/step - loss: 0.2361 - accuracy: 0.9714
Epoch 3/2000
1/1 [==============================] - 2s 2s/step - loss: 0.7813 - accuracy: 0.9126
Epoch 4/2000
1/1 [==============================] - 2s 2s/step - loss: 1.0129 - accuracy: 0.8689
Epoch 5/2000
1/1 [==============================] - 2s 2s/step - loss: 0.8584 - accuracy: 0.8672
Epoch 6/2000
1/1 [==============================] - 2s 2s/step - loss: 0.3770 - accuracy: 0.9361
Epoch 7/2000
1/1 [==============================] - 2s 2s/step - loss: 0.4414 - accuracy: 0.9076
Epoch 8/2000
1/1 [==============================] - 2s 2s/step - loss: 0.3481 - accuracy: 0.9176
Epoch 9/2000
1/1 [==============================] - 3s 3s/step - loss: 0.3063 - accuracy: 0.9160
Epoch 10/2000
1/1 [==============================] - 3s 3s/step - loss: 0.3014 - accuracy: 0.9008
Epoch 11/2000
1/1 [==========

1/1 [==============================] - 3s 3s/step - loss: 0.0262 - accuracy: 0.9916
Epoch 85/2000
1/1 [==============================] - 3s 3s/step - loss: 0.0185 - accuracy: 0.9916
Epoch 86/2000
1/1 [==============================] - 3s 3s/step - loss: 0.0144 - accuracy: 0.9966
Epoch 87/2000
1/1 [==============================] - 3s 3s/step - loss: 0.0156 - accuracy: 0.9950
Epoch 88/2000
1/1 [==============================] - 3s 3s/step - loss: 0.0151 - accuracy: 0.9966
Epoch 89/2000
1/1 [==============================] - 3s 3s/step - loss: 0.0139 - accuracy: 0.9966
Epoch 90/2000
1/1 [==============================] - 3s 3s/step - loss: 0.0114 - accuracy: 1.0000
Epoch 91/2000
1/1 [==============================] - 3s 3s/step - loss: 0.0106 - accuracy: 0.9983
Epoch 92/2000
1/1 [==============================] - 3s 3s/step - loss: 0.0105 - accuracy: 1.0000
Epoch 93/2000
1/1 [==============================] - 3s 3s/step - loss: 0.0342 - accuracy: 0.9882
Epoch 94/2000
1/1 [===============

1/1 [==============================] - 3s 3s/step - loss: 0.4817 - accuracy: 0.8353
Epoch 168/2000
1/1 [==============================] - 3s 3s/step - loss: 0.4690 - accuracy: 0.8420
Epoch 169/2000
1/1 [==============================] - 3s 3s/step - loss: 0.4757 - accuracy: 0.8353
Epoch 170/2000
1/1 [==============================] - 3s 3s/step - loss: 0.4508 - accuracy: 0.8504
Epoch 171/2000
1/1 [==============================] - 3s 3s/step - loss: 0.4317 - accuracy: 0.8504
Epoch 172/2000
1/1 [==============================] - 3s 3s/step - loss: 0.4166 - accuracy: 0.8588
Epoch 173/2000
1/1 [==============================] - 3s 3s/step - loss: 0.4072 - accuracy: 0.8622
Epoch 174/2000
1/1 [==============================] - 3s 3s/step - loss: 0.3903 - accuracy: 0.8622
Epoch 175/2000
1/1 [==============================] - 3s 3s/step - loss: 0.3777 - accuracy: 0.8723
Epoch 176/2000
1/1 [==============================] - 3s 3s/step - loss: 0.3800 - accuracy: 0.8655
Epoch 177/2000
1/1 [=====

1/1 [==============================] - 3s 3s/step - loss: 0.0874 - accuracy: 0.9714
Epoch 250/2000
1/1 [==============================] - 3s 3s/step - loss: 0.0864 - accuracy: 0.9731
Epoch 251/2000
1/1 [==============================] - 3s 3s/step - loss: 0.0824 - accuracy: 0.9714
Epoch 252/2000
1/1 [==============================] - 3s 3s/step - loss: 0.0809 - accuracy: 0.9765
Epoch 253/2000
1/1 [==============================] - 3s 3s/step - loss: 0.0812 - accuracy: 0.9731
Epoch 254/2000
1/1 [==============================] - 3s 3s/step - loss: 0.0766 - accuracy: 0.9731
Epoch 255/2000
1/1 [==============================] - 3s 3s/step - loss: 0.0745 - accuracy: 0.9748
Epoch 256/2000
1/1 [==============================] - 3s 3s/step - loss: 0.0747 - accuracy: 0.9798
Epoch 257/2000
1/1 [==============================] - 3s 3s/step - loss: 0.0712 - accuracy: 0.9765
Epoch 258/2000
1/1 [==============================] - 4s 4s/step - loss: 0.0697 - accuracy: 0.9765
Epoch 259/2000
1/1 [=====

KeyboardInterrupt: 

In [12]:
model.save('action-thai-New-2.h5')

In [92]:
del model

In [21]:
model = load_model('model/best_weights_5.h5')

In [14]:
preds = model.predict(X_test)
preds = np.argmax(preds, axis=1).tolist()
test_labels = np.argmax(y_test, axis=1).tolist()

acc = accuracy_score(test_labels, np.round(preds))*100
cm = confusion_matrix(test_labels, np.round(preds))
Results = ['True' if np.round(preds).flatten()[i] == test_labels[i] else 'False' for i in range(len(preds))]

#hh, ih, th, hi, ii, ti, ht, it, tt = cm.ravel()
#tp = hh
#tn = ii + ti + it + tt
#fp = ih + th
#fn = hi + ht                                           

print('CONFUSION MATRIX ------------------')
print(cm)

print('\nTEST METRICS ----------------------')
#precision = tp/(tp+fp)*100
#recall = tp/(tp+fn)*100
print('Accuracy: {}%'.format(acc))
#print('Precision: {}%'.format(precision))
#print('Recall: {}%'.format(recall))
#print('F1-score: {}'.format(2*precision*recall/(precision+recall)))

print('\nTRAIN METRIC ----------------------')
#print('Train acc: {}'.format(np.round((model.history['accuracy'][-1])*100, 2)))

print(classification_report(y_true=test_labels, y_pred=np.round(preds)))

CONFUSION MATRIX ------------------
[[44  0  0  0  0  0]
 [ 0 36  0  5  0  0]
 [ 0  0 52  0  0  3]
 [ 0  4  1 35  0  2]
 [ 0  0  0  0 40  0]
 [ 0  0  2  3  0 43]]

TEST METRICS ----------------------
Accuracy: 92.5925925925926%

TRAIN METRIC ----------------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        44
           1       0.90      0.88      0.89        41
           2       0.95      0.95      0.95        55
           3       0.81      0.83      0.82        42
           4       1.00      1.00      1.00        40
           5       0.90      0.90      0.90        48

    accuracy                           0.93       270
   macro avg       0.93      0.93      0.93       270
weighted avg       0.93      0.93      0.93       270



In [30]:
preds = model.predict(X_test)
preds = np.argmax(preds, axis=1).tolist()
test_labels = np.argmax(y_test, axis=1).tolist()

acc = accuracy_score(test_labels, np.round(preds))*100
cm = confusion_matrix(test_labels, np.round(preds))
Results = ['True' if np.round(preds).flatten()[i] == test_labels[i] else 'False' for i in range(len(preds))]

print('CONFUSION MATRIX ------------------')
print(cm)

print('\nTEST METRICS ----------------------')

print('Accuracy: {}%'.format(acc))

print('\nTRAIN METRIC ----------------------')

print(classification_report(y_true=test_labels, y_pred=np.round(preds)))

CONFUSION MATRIX ------------------
[[1]]

TEST METRICS ----------------------
Accuracy: 100.0%

TRAIN METRIC ----------------------
              precision    recall  f1-score   support

           6       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1

